# Feature Engineering & Data Preparation

## Purpose

This notebook transforms the FNB customer dataset into model-ready features for churn prediction. The feature engineering draws directly from EDA insights to create predictive signals.

Key transformations:
- Age-based risk indicators (45+ cohort showed 50.6% churn)
- Product complexity scoring (U-shaped pattern: 2 products optimal, 3-4 catastrophic)
- Geographic risk flags (Limpopo, Eastern Cape, Mpumalanga at 32-34% churn)
- Financial ratios and engagement metrics
- Customer lifetime value estimation

## Section 1: Environment Setup

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# FNB colors
fnb_blue = '#003D7A'
fnb_gold = '#FFB81C'

# Settings
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

np.random.seed(42)


## Section 2: Data Loading

Loading the transformed dataset. This is the same data used in EDA, so we know its structure and quality characteristics.

In [2]:
# Load dataset
df = pd.read_csv("Churn_Modelling_SA_FNB.csv")

print(f"Loaded: {df.shape[0]:,} customers, {df.shape[1]} features")
print(f"Baseline churn rate: {df['Churned'].mean():.1%}")
print(f"Class balance: {df['Churned'].value_counts()[0] / df['Churned'].value_counts()[1]:.1f}:1 (retained:churned)")

Loaded: 10,000 customers, 43 features
Baseline churn rate: 20.4%
Class balance: 3.9:1 (retained:churned)


In [3]:
# Verify data quality
print("Missing values check:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("No missing values")
else:
    print(missing[missing > 0])

Missing values check:
No missing values


## Section 3: Feature Engineering Based on EDA Insights

Creating features that capture the churn patterns identified in EDA. Each feature is motivated by a specific finding from the exploratory analysis.

### Age-Based Features

EDA showed age as the strongest predictor (Cohen's d = 0.739). The 46-55 cohort had 50.6% churn vs 20.4% baseline. Creating features to capture this non-linear age effect.

In [4]:
# Age group bins based on EDA findings
df['Age_Group'] = pd.cut(
    df['Age'],
    bins=[0, 25, 35, 45, 55, 65, 100],
    labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+']
)

# High-risk age flag (45+)
df['Is_High_Risk_Age'] = (df['Age'] >= 45).astype(int)

# Age squared for non-linear effects
df['Age_Squared'] = df['Age'] ** 2

print("Age-based features created:")
print(f"  Age_Group distribution:\n{df['Age_Group'].value_counts().sort_index()}")
print(f"\n  High-risk age (45+): {df['Is_High_Risk_Age'].sum():,} customers ({df['Is_High_Risk_Age'].mean():.1%})")

Age-based features created:
  Age_Group distribution:
Age_Group
18-25     611
26-35    3542
36-45    3736
46-55    1311
56-65     536
65+       264
Name: count, dtype: int64

  High-risk age (45+): 2,340 customers (23.4%)


### Product Complexity Features

EDA revealed U-shaped churn: 1 product (27.7%), 2 products (7.6%), 3 products (82.7%), 4 products (100%). Creating features to flag these patterns.

In [5]:
# Single product flag (under-engaged)
df['Is_Single_Product'] = (df['NumOfProducts'] == 1).astype(int)

# Optimal products flag (2 products)
df['Is_Optimal_Products'] = (df['NumOfProducts'] == 2).astype(int)

# Over-banked flag (3-4 products)
df['Is_Over_Banked'] = (df['NumOfProducts'] >= 3).astype(int)

# Product complexity score
# Higher score for extremes (1 or 4 products), lower for optimal (2)
product_risk_map = {1: 2.0, 2: 0.5, 3: 3.0, 4: 4.0}
df['Product_Risk_Score'] = df['NumOfProducts'].map(product_risk_map)

print("Product complexity features:")
print(f"  Single product: {df['Is_Single_Product'].sum():,} customers")
print(f"  Optimal (2): {df['Is_Optimal_Products'].sum():,} customers")
print(f"  Over-banked (3-4): {df['Is_Over_Banked'].sum():,} customers")

Product complexity features:
  Single product: 5,084 customers
  Optimal (2): 4,590 customers
  Over-banked (3-4): 326 customers


### Financial Profile Features

EDA showed churned customers have higher balances (R1.87M vs R1.49M). Creating ratio features and wealth indicators.

In [6]:
# Balance to salary ratio
df['Balance_to_Salary_Ratio'] = df['Balance_ZAR'] / (df['AnnualSalary_ZAR'] + 1)  # +1 to avoid division by zero

# Zero balance flag
df['Has_Zero_Balance'] = (df['Balance_ZAR'] == 0).astype(int)

# High value customer (balance > R2M)
df['Is_High_Value'] = (df['Balance_ZAR'] >= 2_000_000).astype(int)

# Balance quartiles
try:
    df['Balance_Quartile'] = pd.qcut(df['Balance_ZAR'].rank(method='first'), q=4, labels=False, duplicates='drop') + 1
except ValueError:
    # Fallback if qcut fails due to duplicate edges
    df['Balance_Quartile'] = pd.cut(df['Balance_ZAR'], bins=4, labels=False) + 1

print("Financial features created:")
print(f"  Zero balance: {df['Has_Zero_Balance'].sum():,} customers")
print(f"  High value (R2M+): {df['Is_High_Value'].sum():,} customers")
print(f"  Balance quartile distribution:\n{df['Balance_Quartile'].value_counts().sort_index()}")

Financial features created:
  Zero balance: 3,617 customers
  High value (R2M+): 4,968 customers
  Balance quartile distribution:
Balance_Quartile
1    2500
2    2500
3    2500
4    2500
Name: count, dtype: int64


### Engagement Features

EDA showed inactive members churn at 26.9% vs 14.3% for active. Digital adoption showed minimal impact, but creating engagement composite anyway.

In [7]:
# Digital adoption score (0-2)
df['Digital_Adoption_Score'] = df['UsesMobileBanking'] + df['UsesInternetBanking']

# Digitally engaged flag
df['Is_Digitally_Engaged'] = (df['Digital_Adoption_Score'] == 2).astype(int)

# Service friction score (complaints + high service calls)
df['Service_Friction_Score'] = df['Complaints_12M'] + (df['CustomerServiceCalls_12M'] > 3).astype(int)

# Has complaints flag
df['Has_Complaints'] = (df['Complaints_12M'] > 0).astype(int)

print("Engagement features:")
print(f"  Digitally engaged (both channels): {df['Is_Digitally_Engaged'].sum():,} customers")
print(f"  Has complaints: {df['Has_Complaints'].sum():,} customers")

Engagement features:
  Digitally engaged (both channels): 5,602 customers
  Has complaints: 3,907 customers


### Tenure-Based Features

EDA showed tenure doesn't differentiate churn (19-23% across all cohorts), but including for completeness and potential interactions.

In [8]:
# Tenure quartiles
try:
    df['Tenure_Quartile'] = pd.qcut(df['Tenure'], q=4, labels=False, duplicates='drop') + 1
except ValueError:
    df['Tenure_Quartile'] = pd.cut(df['Tenure'], bins=4, labels=False) + 1

# New customer flag (tenure <= 1 year)
df['Is_New_Customer'] = (df['Tenure'] <= 1).astype(int)

# Tenure to age ratio (relationship maturity relative to age)
df['Tenure_to_Age_Ratio'] = df['Tenure'] / (df['Age'] + 1)

print("Tenure features:")
print(f"  New customers (<=1 year): {df['Is_New_Customer'].sum():,}")
print(f"  Tenure quartile distribution:\n{df['Tenure_Quartile'].value_counts().sort_index()}")

Tenure features:
  New customers (<=1 year): 1,448
  Tenure quartile distribution:
Tenure_Quartile
1    3505
2    2001
3    1995
4    2499
Name: count, dtype: int64


### Geographic Risk Features

EDA showed Limpopo (34%), Eastern Cape (32%), Mpumalanga (31%) with elevated churn vs Gauteng/KZN at 15%.

In [9]:
# High-risk province flag
high_risk_provinces = ['Limpopo', 'Eastern Cape', 'Mpumalanga']
df['Is_High_Risk_Province'] = df['Province'].isin(high_risk_provinces).astype(int)

print("Geographic features:")
print(f"  High-risk province customers: {df['Is_High_Risk_Province'].sum():,}")
print(f"  High-risk provinces: {', '.join(high_risk_provinces)}")

Geographic features:
  High-risk province customers: 2,509
  High-risk provinces: Limpopo, Eastern Cape, Mpumalanga


### Credit Risk Features

EDA showed credit score has minimal impact (Cohen's d = -0.067), but creating tiers for completeness.

In [10]:
# Credit score tiers
df['Credit_Risk_Tier'] = pd.cut(
    df['CreditScore'],
    bins=[0, 600, 650, 700, 850],
    labels=['Poor', 'Fair', 'Good', 'Excellent']
)

# Poor credit flag
df['Is_Poor_Credit'] = (df['CreditScore'] < 600).astype(int)

print("Credit features:")
print(f"  Poor credit (<600): {df['Is_Poor_Credit'].sum():,} customers")
print(f"  Credit tier distribution:\n{df['Credit_Risk_Tier'].value_counts().sort_index()}")

Credit features:
  Poor credit (<600): 3,034 customers
  Credit tier distribution:
Credit_Risk_Tier
Poor         3066
Fair         1871
Good         1947
Excellent    3116
Name: count, dtype: int64


### Composite Risk Score

Creating a composite risk score based on validated churn drivers from Advanced EDA. This mirrors the rule-based score but will compete with ML models.

In [11]:
# Composite risk score (weighted sum of risk factors)
df['Composite_Risk_Score'] = (
    df['Is_High_Risk_Age'] * 3.0 +                # Age 45+ (strongest predictor)
    df['HasPersonalLoan'] * 2.5 +                 # Personal loan (85.9% churn)
    df['Is_Over_Banked'] * 2.5 +                  # 3-4 products (83-100% churn)
    (1 - df['IsActiveMember']) * 2.0 +            # Inactive (26.9% vs 14.3%)
    df['Is_High_Risk_Province'] * 1.5 +           # Limpopo/EC/Mpumalanga
    df['Is_High_Value'] * 1.0                     # High balance (counter-intuitive but validated)
)

print("Composite Risk Score:")
print(f"  Mean: {df['Composite_Risk_Score'].mean():.2f}")
print(f"  Median: {df['Composite_Risk_Score'].median():.2f}")
print(f"  Range: {df['Composite_Risk_Score'].min():.1f} to {df['Composite_Risk_Score'].max():.1f}")

Composite Risk Score:
  Mean: 2.71
  Median: 2.50
  Range: 0.0 to 12.5


### Customer Lifetime Value (CLV)

Estimating CLV for business impact analysis. Using simplified model: tenure-based revenue + balance-based margin.

In [12]:
# CLV estimation
avg_annual_revenue = 50_000  # Conservative estimate per customer
balance_margin = 0.02        # 2% margin on deposits

df['Monthly_Revenue'] = avg_annual_revenue / 12
df['Estimated_CLV_5yr'] = (
    avg_annual_revenue * 5 +  # 5-year revenue horizon
    df['Balance_ZAR'] * balance_margin
)

print("CLV Estimation:")
print(f"  Mean CLV (5-year): R{df['Estimated_CLV_5yr'].mean():,.0f}")
print(f"  Total customer base CLV: R{df['Estimated_CLV_5yr'].sum()/1_000_000:,.1f}M")

CLV Estimation:
  Mean CLV (5-year): R281,359
  Total customer base CLV: R2,813.6M


## Section 4: Feature Selection

Identifying which features to use for modeling. Excluding identifiers, redundant features, and zero-variance features.

In [13]:
# Features to exclude
exclude_features = [
    # Identifiers
    'RowNumber', 'CustomerId', 'Surname',
    # Zero variance
    'Bank', 'CreditBureau',
    # Redundant
    'HasCreditCard_Flag',  # Duplicate of HasCreditCard
    'AgeBand',  # Using Age_Group instead
    'MonthlySalary_ZAR',  # Using AnnualSalary_ZAR
    # High cardinality with minimal value
    'City',  # Province captures geographic risk better
    # Target variable
    'Churned'
]

# Separate numerical and categorical features
all_features = [col for col in df.columns if col not in exclude_features]

numerical_features = df[all_features].select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df[all_features].select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Total features for modeling: {len(all_features)}")
print(f"  Numerical: {len(numerical_features)}")
print(f"  Categorical: {len(categorical_features)}")

print(f"\nNumerical features ({len(numerical_features)}):")
for feat in numerical_features[:10]:
    print(f"  - {feat}")
if len(numerical_features) > 10:
    print(f"  ... and {len(numerical_features) - 10} more")

print(f"\nCategorical features ({len(categorical_features)}):")
for feat in categorical_features:
    print(f"  - {feat} ({df[feat].nunique()} unique values)")

Total features for modeling: 57
  Numerical: 44
  Categorical: 13

Numerical features (44):
  - Age
  - Tenure
  - CreditScore
  - NumOfProducts
  - HasSavingsAccount
  - HasChequeAccount
  - HasCreditCard
  - HasPersonalLoan
  - HasHomeLoan
  - HasVehicleFinance
  ... and 34 more

Categorical features (13):
  - Gender (2 unique values)
  - Province (9 unique values)
  - AreaType (3 unique values)
  - EducationLevel (5 unique values)
  - EmploymentType (5 unique values)
  - PreferredLanguage (11 unique values)
  - AccountType (5 unique values)
  - CreditRating (5 unique values)
  - ProductName (4 unique values)
  - SalaryPaymentMethod (3 unique values)
  - PreferredCommChannel (5 unique values)
  - Age_Group (6 unique values)
  - Credit_Risk_Tier (4 unique values)


## Section 5: Train-Test Split

Splitting data before any preprocessing to avoid data leakage. Using stratified split to maintain class balance.

In [14]:
# Prepare X and y
X = df[all_features].copy()
y = df['Churned'].copy()

# Stratified split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train-Test Split:")
print(f"  Training set: {len(X_train):,} samples ({len(X_train)/len(X):.1%})")
print(f"  Test set: {len(X_test):,} samples ({len(X_test)/len(X):.1%})")

print(f"\nChurn rate preserved:")
print(f"  Training: {y_train.mean():.1%}")
print(f"  Test: {y_test.mean():.1%}")
print(f"  Original: {y.mean():.1%}")

Train-Test Split:
  Training set: 8,000 samples (80.0%)
  Test set: 2,000 samples (20.0%)

Churn rate preserved:
  Training: 20.4%
  Test: 20.3%
  Original: 20.4%


## Section 6: Preprocessing Pipeline

Building a scikit-learn pipeline for consistent transformation of numerical and categorical features. This ensures the same transformations apply to train, test, and future scoring.

In [15]:
from sklearn.preprocessing import OneHotEncoder

# Numerical pipeline: impute missing + scale
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline: impute missing + one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

# Combined preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

print("Preprocessing pipeline created")
print(f"  Numerical features: {len(numerical_features)} (impute + scale)")
print(f"  Categorical features: {len(categorical_features)} (impute + one-hot encode)")

Preprocessing pipeline created
  Numerical features: 44 (impute + scale)
  Categorical features: 13 (impute + one-hot encode)


In [16]:
# Fit preprocessor on training data only
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Data preprocessed:")
print(f"  Training shape: {X_train_processed.shape}")
print(f"  Test shape: {X_test_processed.shape}")
print(f"  Total features after encoding: {X_train_processed.shape[1]}")

Data preprocessed:
  Training shape: (8000, 98)
  Test shape: (2000, 98)
  Total features after encoding: 98


## Section 7: Save Prepared Data

Saving the processed data and preprocessing pipeline for the modeling notebook. Also saving feature metadata for reference.

In [17]:
import joblib

# Save preprocessed data
np.save('X_train_processed.npy', X_train_processed)
np.save('X_test_processed.npy', X_test_processed)
np.save('y_train.npy', y_train.values)
np.save('y_test.npy', y_test.values)

# Save preprocessor
joblib.dump(preprocessor, 'preprocessor.pkl')

# Save feature names
feature_metadata = {
    'numerical_features': numerical_features,
    'categorical_features': categorical_features,
    'all_features': all_features
}
joblib.dump(feature_metadata, 'feature_metadata.pkl')

# Save original test set with IDs for business analysis
test_indices = X_test.index
test_data_with_target = df.loc[test_indices, ['CustomerId', 'Estimated_CLV_5yr', 'Churned']].copy()
test_data_with_target.to_csv('test_set_metadata.csv', index=True)

print("Saved:")
print("  ✓ X_train_processed.npy")
print("  ✓ X_test_processed.npy")
print("  ✓ y_train.npy")
print("  ✓ y_test.npy")
print("  ✓ preprocessor.pkl")
print("  ✓ feature_metadata.pkl")
print("  ✓ test_set_metadata.csv")

Saved:
  ✓ X_train_processed.npy
  ✓ X_test_processed.npy
  ✓ y_train.npy
  ✓ y_test.npy
  ✓ preprocessor.pkl
  ✓ feature_metadata.pkl
  ✓ test_set_metadata.csv


In [18]:
# Save the engineered dataset with all features
df.to_csv('FNB_Churn_Engineered_Features.csv', index=False)

print("✓ Saved: FNB_Churn_Engineered_Features.csv")
print(f"  Rows: {df.shape[0]:,}")
print(f"  Columns: {df.shape[1]}")
print(f"  Original features: 43")
print(f"  Total features: {df.shape[1]}")
print(f"  New engineered features: {df.shape[1] - 43}")

✓ Saved: FNB_Churn_Engineered_Features.csv
  Rows: 10,000
  Columns: 67
  Original features: 43
  Total features: 67
  New engineered features: 24


## Summary

**Features Created**: 25+ engineered features based on EDA insights

**Key Feature Groups**:
- Age risk indicators (strongest predictor)
- Product complexity scores (U-shaped pattern)
- Geographic risk flags (provincial concentration)
- Financial ratios and wealth indicators
- Engagement and activity metrics
- Composite risk score
- Customer lifetime value estimation

**Data Prepared**:
- Training set: 8,000 customers (20.4% churn)
- Test set: 2,000 customers (20.4% churn)
- Features: 60+ after one-hot encoding

**Next Steps**: Model development in Notebook 04 using these engineered features.